**라이브러리 임포트**

In [1]:
import warnings

# 특정 모듈에서 발생하는 FutureWarning 무시
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r"You are using `torch.load` with `weights_only=False`",
)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install import_ipynb
import import_ipynb

from torchvision import models
import torch

import os
import unicodedata

In [19]:
%cd '/content/drive/MyDrive/CAB'

/content/drive/MyDrive/CAB


**사전 학습된 모델 로드**

이전에 학습시킨 ResNet50 모델을 불러온다.

In [5]:
from Get_State_Using_ResNet50 import crop_image, load_and_process_image, predict, imshow, MyResNet50

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 149MB/s]


Predicted [Left Eye, Right Eye, Mouth]: [Open, Open, Open]


In [6]:
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MyResNet50(resnet50)
model.load_state_dict(torch.load('/content/drive/MyDrive/CAB/CAB_dataset/model/best_resnet_model.pth', map_location=torch.device('cpu')))
model.to(device);

이전에 학습시킨 YOLOv5 모델을 불러온다.

In [21]:
!git clone https://github.com/jnalgae/CustomizedYOLOv5
%cd CustomizedYOLOv5

Cloning into 'CustomizedYOLOv5'...
remote: Enumerating objects: 11435, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 11435 (delta 50), reused 17 (delta 8), pack-reused 11350 (from 1)
Receiving objects: 100% (11435/11435), 13.34 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (7595/7595), done.
Updating files: 100% (144/144), done.
/content/drive/MyDrive/CAB/CustomizedYOLOv5


In [22]:
# import importlib
import detect
from detect import run
# importlib.reload(detect)

def img_detect(source_path, img_size=(480, 480), weights_path="/content/drive/MyDrive/CAB/CAB_dataset/model/best_YOLO_model.pt",):
  return run(weights=weights_path, imgsz=img_size, source=source_path, conf_thres=0.45)

**inference 수행**

추론 단계에 앞서 각 class와 매칭되는 label을 정의한다.

In [23]:
cls = {
    "흡연": 0,
    "통화": 1,
    "졸음": 2,
    "하품": 3,
    "정상": 4
}

inference 함수를 정의한다. 담배 혹은 휴대전화의 유무, 양쪽 눈과 입의 상태 정보(open/close)를 이용하여 이미지가 해당하는 class(흡연/통화/졸음/하품/정상)를 판단한다.

In [33]:
def inference(path):
  is_drowsiness, is_yawn = False, False

  is_cigar, is_phone, leye, reye, mouth = img_detect(path)

  if leye != [0, 0, 0, 0]: # leye object를 detect 했을 경우
    leye_img = crop_image(path, leye) # yolo가 추론한 좌표를 바탕으로 이미지를 자름
    predicted_leye = predict(model, leye_img) # 자른 이미지를 바탕으로 에측(open/close) 수행
  else:
    predicted_leye = None  # leye object를 detect 하지 못했을 경우

  if reye != [0, 0, 0, 0]:
    reye_img = crop_image(path, reye)
    predicted_reye = predict(model, reye_img)
  else:
    predicted_reye = None

  if mouth != [0, 0, 0, 0]:
    mouth_img = crop_image(path, mouth)
    predicted_mouth = predict(model, mouth_img)
  else:
     predicted_mouth = None

  if predicted_leye == "Close" and predicted_reye == "Close": # 양쪽 눈이 모두 감겨 있는 경우
    is_drowsiness = True

  if predicted_mouth == "Open": # 입이 열려 있을 경우
    is_yawn = True

  if is_cigar:
    return cls["흡연"] # 각 클래스와 매칭되는 label 반환

  elif is_phone:
    return cls["통화"]

  elif is_drowsiness:
    return cls["졸음"]

  elif is_yawn:
    return cls["하품"]

  else:
    return cls["정상"]

target을 저장할 true_labels 리스트와 예측 결과를 저장할 predicted_labels 리스트를 생성한다.

target은 이미지 파일명을 확인하여 지정하였다. 예를 들어, 이미지 파일명이 졸음재현.jpg일 경우 target은 졸음에 해당하는 label, 2이다. 이때 파일명에 흡연/통화/졸음/하품/정상 중 어떤 단어가 있는지 찾는 과정에서 문자열간 정규화 방식을 통일하기 위해 'NFC' 방식(자음 모음 결합)'을 사용하였다.

In [35]:
true_labels = []
predicted_labels = []

test_root = "/content/drive/MyDrive/CAB/CAB_dataset/test_no_txt/"

count = 0

for file in os.listdir(test_root):
  path = os.path.join(test_root, file) # 이미지 path 생성

  true_label = None

  for state, value in cls.items():
    if unicodedata.normalize("NFC", state) in unicodedata.normalize("NFC", file): # 원활한 한글 비교를 위해 NFC 사용
      true_label = value
      break

  result = inference(path)

  true_labels.append(true_label)
  predicted_labels.append(result)

y_true_tensor = torch.tensor(true_labels)
y_pred_tensor = torch.tensor(predicted_labels)

각 클래스별 Precision, Recall, F1 Score과 전체 Accuracy 확인하는 과정이다.

In [36]:
for _, value in cls.items():
  TP = ((y_pred_tensor == value) & (y_true_tensor == value)).sum().item()
  FP = ((y_pred_tensor == value) & (y_pred_tensor != value)).sum().item()
  FN = ((y_pred_tensor != value) & (y_true_tensor == value)).sum().item()

  precision = TP / (TP + FP) if (TP + FP) > 0 else 0
  recall = TP / (TP + FN) if (TP + FN) > 0 else 0
  f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

  print(f"Class {value}: Precision={precision:.2f}, Recall={recall:.2f}, F1 Score={f1:.2f}")

accuracy = ((y_pred_tensor == y_true_tensor).sum().item()) / len(y_pred_tensor)
print(f"Accuracy: {accuracy}")

Class 0: Precision=1.00, Recall=0.97, F1 Score=0.99
Class 1: Precision=1.00, Recall=0.99, F1 Score=1.00
Class 2: Precision=1.00, Recall=0.73, F1 Score=0.84
Class 3: Precision=1.00, Recall=0.86, F1 Score=0.92
Class 4: Precision=1.00, Recall=0.90, F1 Score=0.95
Accuracy: 0.8726114649681529
